# Stage 0: Load the Libraries

In [165]:
# Stage 0: import Libraries
import pandas
import numpy
import requests
import json
import boto3
from langchain_aws import BedrockLLM
from pypdf import PdfReader
from langchain_community.chat_models import BedrockChat
from langchain_core.tools import tool

from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent
from langchain_aws import ChatBedrock


# Stage 1: AWS Configuration and Define Global Variables


In [147]:
llm = BedrockLLM(
    model_id="mistral.mixtral-8x7b-instruct-v0:1",
    model_kwargs = {"temperature": 0, "max_tokens": 4000, "top_p": 0.1, "top_k": 50}
)

model_id = "anthropic.claude-3-haiku-20240307-v1:0"
model_kwargs = {"temperature": 0, "max_tokens": 4000, "top_p": 0.1, "top_k": 50}
chat = BedrockChat(model_id=model_id, model_kwargs=model_kwargs)


In [51]:
chat.invoke("Hey , my name is Anshumaan!").content

"It's nice to meet you, Anshumaan! I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you might have. Please let me know if there's anything I can assist you with."

In [148]:
resume_reader = PdfReader(RESUME_FILE_PATH)
resume_text = ""

for page in resume_reader.pages:
    resume_text += page.extract_text()


resume_text

"ANSHUMAAN TYAGI\nHP: +65 80169372      Email: e0866249@u.nus.edu\nEDUCATION\nNational University of Singapore Aug 2021 - May 2025\nMajor: Bachelor of Computing in Computer Science(Honours)\n\uf0b7Second Major: Data Science.\n\uf0b7Specialisation Track: Software Engineering and Artificial Intelligence.\n\uf0b7GPA: 4.31/5.\n \nTECHNICAL SKILLS\n\uf0b7Proficient languages: Python, Java ,SQL,  Javascript, HTML and CSS\n\uf0b7Proficient frameworks/libraries: Pytorch, Numpy, Django, PostgreSQL, React & React Native,Node.js, \nSklearn, Firebase, Excel\n\uf0b7Knowledge of: MongoDB, LINUX/UNIX administration , R, Azure Open AI, GPT prompting, \nembeddings, Spacy\n \nWORK EXPERIENCE\nAI Developer, SAP, Singapore Jan 2023 - Present\n\uf0b7Integrating GPT powered by Azure Open AI endpoints in the current pipeline to perform parsing of \nPDF documents.\n\uf0b7Improving GPT response efficiency by 5 percent and data security.\n\uf0b7Utilized different prompting techniques such as RAG and chain of th

In [149]:
#  Define AWS Global Variables
AWS_KB_ID = "OY4K3P2JUR"
client = boto3.client('bedrock-agent-runtime')




In [176]:
# Define Aspiration
ASPIRATION = "I want to be a better software engineer"


In [55]:
context = client.retrieve(knowledgeBaseId=AWS_KB_ID, retrievalQuery={
        'text': ASPIRATION
    })

# Stage 2: Query the Knowledge Base and Transform the context

In [56]:
response = client.retrieve(
    knowledgeBaseId=AWS_KB_ID,
     retrievalQuery={
        'text': ASPIRATION,
    }
)




# Stage 3: Prompt Design & Data Models

In [34]:
# Stage 3: Prompt Design & Data Models

skills_extraction_prompt = """
You are Career Consultant. You are tasked with extracting skills required to reach the aspiration below.

Task:
1. Understand the Job Descriptions below relevant to the aspiration.
2. From your Undestanding, extract all the skill entities required to reach the aspiration.
3. For each skill,be specfic, provide the actual skill extracted, do not combine skills together and list them out individually.
4. Also provide justification of the reason why the skills is needed to reach the aspiration.
5. For each skill level, be strict , Beginner: 0-1 years, Intermediate: 1-3 years, Advanced: 3-5 years, Expert: 5+ years.
4. For each skill level, provide a justification of how the aspiration requires a skill of that level.


FORMAT Response Instruction:
----------------
{format_instructions}
----------------


----------------
aspiration: {aspiration}
----------------
----------------
job descriptions: {job_descriptions}
----------------



"""

fix_format_instruction = """
--------------
{instructions}
--------------
Completion:
--------------
{completion}
--------------
Above, the Completion did not satisfy the constraints given in the Instructions.
Error:
--------------
{error}
--------------
Please try again. Please only respond with an answer that satisfies the constraints laid out in the Instructions.
Important:  Only correct the structural issues within the JSON format. Do not modify the existing data values themselves:
"""

In [35]:
# Data Models
from typing import List
from pydantic import BaseModel, Field

# class Competency(BaseModel):
#     competency_type: str = Field(..., description="Type of competency:Functional, Technical")
#     competency_name: str = Field(..., description="Name of the competency")
#     competency_description: str = Field(..., description="Description of how the client has demonstrated the competency in their resume")

# class Comptencies(BaseModel):
#     competencies: List[Competency] = Field(..., description="List of competencies extracted from the resume")

class HardSkill(BaseModel):
    skill: str = Field(..., description="Name of the skill")
    skill_explanation: str = Field(..., description="explanation of skill demonstration using context from the text and the context's contribution to the skill")#

class HardSkills(BaseModel):
    hard_skills: List[HardSkill] = Field(..., description="List of hard skills ")
    


# Stage 4: Extract Skills required for aspiration

In [150]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser
from langchain.chains import TransformChain

def fix_chain_fun(inputs):    
    fix_prompt = PromptTemplate.from_template(fix_format_instruction)
    fix_prompt_str = fix_prompt.invoke({'instructions':inputs['instructions'],
                                        'completion':inputs['completion'],
                                        'error':inputs['error']}).text
    
    #print(fix_prompt_str)
    
    completion = llm.invoke(fix_prompt_str)

    return {"completion": completion}

fix_chain = TransformChain(
    input_variables = ["instructions", "completion", "error"],output_variables=["completion"], transform=fix_chain_fun
)

def extract_long_time_goal(aspiration, job_descriptions: str) -> Skills:
    # Invoke the LLM
    parser = PydanticOutputParser(pydantic_object=Skills)
    
    fix_parser = OutputFixingParser(
        parser=parser,
        retry_chain=fix_chain,
        max_retries=2
    )

    prompt = PromptTemplate(
        template = skills_extraction_prompt, 
        input_variables=["aspiration", "job_descriptions"],
        partial_variables= {"format_instructions": parser.get_format_instructions()})
    
    prompt_str = prompt.format(aspiration=aspiration, job_descriptions=job_descriptions)

    print(prompt_str)
    
    response = llm.invoke(prompt_str)
    response = response

    print(f"Response is : {response}")

    fixed_response = fix_parser.invoke(response).dict()

    return fixed_response



In [37]:
aspiration_extracted_skills = extract_long_time_goal(ASPIRATION, context)


You are Career Consultant. You are tasked with extracting skills required to reach the aspiration below.

Task:
1. Understand the Job Descriptions below relevant to the aspiration.
2. From your Undestanding, extract all the skill entities required to reach the aspiration.
3. For each skill,be specfic, provide the actual skill extracted, do not combine skills together and list them out individually.
4. Also provide justification of the reason why the skills is needed to reach the aspiration.
5. For each skill level, be strict , Beginner: 0-1 years, Intermediate: 1-3 years, Advanced: 3-5 years, Expert: 5+ years.
4. For each skill level, provide a justification of how the aspiration requires a skill of that level.


FORMAT Response Instruction:
----------------
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type"

In [38]:
with open("aspiration_extracted_skills.json", "w") as f:
    json.dump(aspiration_extracted_skills, f, indent=4) 

# Stage 5: Aspiration Skill Agent

In [166]:
TARGET_SKILLS = {}
LEARNING_GOAL = None
ASPIRATION = "I want to be a better software engineer"

In [167]:
# Define the prompts
aspiration_extraction_prompt = """
You are a career consellor. 
You are translating client's aspiration into a SMART goal. 
Task: 
1. Understand SMART Goals' Definition and Criteria. 
2. Understand client Resume. 
3. Understand given example , with client aspiration translated into SMART goal. 
4. Translate client's aspiration into a SMART goal, explain the translation. 

Disclaimer: Translate the goals as it is , use the resume to infer information. 
------------------ 
Domain 
SMART Goal Defintion: Statements of the important results you are working to accomplish.Designed in a way to foster clear and mutual understanding of what constitutes expected levels of performance and successful professional development 
SMART Goal Ceriteria: 
    S-> Specific: What will be accomplished? What actions will you take? 
    M-> Measurable: What data will measure the goal? (How much? How well? 
    A->Achievable: Is the goal doable? Do you have the necessary skills and resources? 
    R->Relevant: How does the goal align with broader goals? Why is the result important? 
    T->Time-Bound: What is the time frame for accomplishing the goal? 

------------------
Resume: {resume}  
------------------ 

------------------
Example: 
Aspiration: I want to improve my performance  
GOAL: 
    Specific: I received low marks on my ability to use PowerPoint at my last performance review. Improving my skills requires that I learn how to use PowerPoint efficiently and practice using it by creating various presentations. I’d like to be more proficient using PowerPoint in time for my next review in six months. 
    Measurable: By the time of my next review, I should be able to create presentations that incorporate graphs, images, and other media in a couple of hours. I should also be able to efficiently use and create templates in PowerPoint that my coworkers can also use. 
    Achievable: Improving my PowerPoint skills is instrumental in moving forward in my career and receiving a better performance review. I can set time aside every week to watch PowerPoint tutorials and even enroll in an online class that can teach me new skills. I can also ask coworkers and my manager for PowerPoint tips. 
    Relevant: Working with PowerPoint is currently 25% of my job. As I move up in the company, I’ll need to spend 50% of my time creating PowerPoint presentations. I enjoy my career and want to continue to grow within this company. 
    Time-Bound: In six months, I should be proficient in PowerPoint ensuring it only occupies 25% of my workload instead of the nearly 40% of the time it occupies now. 

------------------ 

------------------ 
Aspiration: {aspiration}
------------------ 

------------------ 
format_instruction: {format_instructions} 
------------------

"""

skill_entity_extraction_prompt = """
You are a career counselor. Your task is to extract skill entities from the given text, which can be a job description.

Skill Entities:
Skills: Also called technical skills, these are job-specific and relevant to each position and seniority level. In other words, each position in every company will require unique hard skills.

TASK:
1. Perform a Part-of-Speech (POS) Tagging on the text.
2. Using the POS-tagged text, perform Name Entity Recognition to identify all explicitly stated skill entities.
3. For each skill, provide an explanation of skill demonstration using context from the text and the skill's contribution to the job description.
4. State the context as it appears in the text, without extrapolating it with other skills.

----------------------
Format Instruction:
{format_instructions}
----------------------

----------------------
Text: {text}
----------------------
"""




fix_format_instruction = """
--------------
{instructions}
--------------
Completion:
--------------
{completion}
--------------
Above, the Completion did not satisfy the constraints given in the Instructions.
Error:
--------------
{error}
--------------
Please try again. Please only respond with an answer that satisfies the constraints laid out in the Instructions.
Important:  Only correct the structural issues within the JSON format. Do not modify the existing data values themselves:
"""


In [168]:
# Data Models
from typing import List
from pydantic import BaseModel, Field


class Goal(BaseModel):
    specific: str = Field(..., description="Specific: What will be accomplished? What actions will you take?")
    measurable: str = Field(..., description="Measurable: What data will measure the goal? (How much? How well?")
    achievable: str = Field(..., description="Achievable: Is the goal doable? Do you have the necessary skills and resources?")
    relevant: str = Field(..., description="Relevant: How does the goal align with broader goals? Why is the result important?")
    time_bound: str = Field(..., description="Time-Bound: What is the time frame for accomplishing the goal?")
    explanation: str = Field(..., description="Explanation of the translation of the aspiration into a SMART goal")

class Skill(BaseModel):
    skill: str = Field(..., description="Name of the skill")
    skill_explanation: str = Field(..., description="explanation of skill demonstration using context from the text and the context's contribution to the skill")#

class Skills(BaseModel):
    skills: List[Skill] = Field(..., description="List of skills")



In [173]:
# Function to extract the SMART goal


def fix_chain_fun(inputs):    
    fix_prompt = PromptTemplate.from_template(fix_format_instruction)
    fix_prompt_str = fix_prompt.invoke({'instructions':inputs['instructions'],
                                        'completion':inputs['completion'],
                                        'error':inputs['error']}).text
    
    #print(fix_prompt_str)
    
    completion = chat.invoke(fix_prompt_str).content

    return {"completion": completion}

fix_chain = TransformChain(
    input_variables = ["instructions", "completion", "error"],output_variables=["completion"], transform=fix_chain_fun
)

@tool(parse_docstring=True)
def extract_smart_goal(aspiration: str, resume: str) -> Goal:
    """Generates a SMART goal from the user's aspiration and resume.Takes User Aspiration and Resume as input and returns a SMART goal.

    Args:
        aspiration: User's Aspiration
        resume: User's Resume
    """
    parser = PydanticOutputParser(pydantic_object=Goal)
    
    fix_parser = OutputFixingParser(
        parser=parser,
        retry_chain=fix_chain,
        max_retries=2
    )

    prompt = PromptTemplate(
        template = aspiration_extraction_prompt, 
        input_variables=["aspiration", "resume"],
        partial_variables= {"format_instructions": parser.get_format_instructions()})
    
    prompt_str = prompt.format(aspiration=aspiration, resume=resume)

    print(prompt_str)
    
    response = chat.invoke(prompt_str).content
 

    print(f"Response is : {response}")

    fixed_response = fix_parser.invoke(response).dict()

    LEARNING_GOAL = fixed_response

    return fixed_response

# A tool consists of name of the tool, Description of the tool , A JSON schema defining the inputs to the tool and A function

def extract_skills_context(job_desc):
    parser = PydanticOutputParser(pydantic_object= Skills)
    
    fix_parser = OutputFixingParser(
        parser=parser,
        retry_chain=fix_chain,
        max_retries=2
    )

    prompt = PromptTemplate(
        template = skill_entity_extraction_prompt, 
        input_variables=["text"],
        partial_variables= {"format_instructions": parser.get_format_instructions()})
    
    prompt_str = prompt.format(text=job_desc)

    print(prompt_str)
    
    response = llm.invoke(prompt_str)


    print(f"Response is : {response}")

    fixed_response = fix_parser.invoke(response).dict()

    for skill_set in fixed_response["skills"]:
        
        key = skill_set["skill"].lower()
        justification = skill_set["skill_explanation"]

        TARGET_SKILLS[key] = justification

    return str(fixed_response) + "Skills were extracted successfully!"

@tool(parse_docstring=True)
def get_skills(specific_goal: str) ->  str:
    """Runs on User's explicit instruction to extracts skills from the learning goal.Take's in the specific goal and returns the extracted skills.
    
    Args:
        specific_goal: Resume String for skill extraction.
    """
    # Invoke the LLM
    context = response = client.retrieve(
    knowledgeBaseId=AWS_KB_ID,
     retrievalQuery={
        'text': specific_goal,
    }
)
    responses = ""
    for result in context["retrievalResults"]:
        job_desc = result["content"]["text"]
        print(job_desc)
        responses += extract_skills_context(job_desc)
    
    return responses
    
    
    



get_skills.args_schema.schema()
# extract_smart_goal.args_schema.schema()

{'title': 'get_skillsSchema',
 'description': "Runs on User's explicit instruction to extracts skills from the learning goal.Take's in the specific goal and returns the extracted skills.",
 'type': 'object',
 'properties': {'specific_goal': {'title': 'Specific Goal',
   'description': 'Resume String for skill extraction.',
   'type': 'string'}},
 'required': ['specific_goal']}

### Stage 3: Testing the Functions

In [177]:
ASPIRATION = "I want to be a better software engineer"
learning_plan = extract_smart_goal(ASPIRATION, resume_text)



You are a career consellor. 
You are translating client's aspiration into a SMART goal. 
Task: 
1. Understand SMART Goals' Definition and Criteria. 
2. Understand client Resume. 
3. Understand given example , with client aspiration translated into SMART goal. 
4. Translate client's aspiration into a SMART goal, explain the translation. 

Disclaimer: Translate the goals as it is , use the resume to infer information. 
------------------ 
Domain 
SMART Goal Defintion: Statements of the important results you are working to accomplish.Designed in a way to foster clear and mutual understanding of what constitutes expected levels of performance and successful professional development 
SMART Goal Ceriteria: 
    S-> Specific: What will be accomplished? What actions will you take? 
    M-> Measurable: What data will measure the goal? (How much? How well? 
    A->Achievable: Is the goal doable? Do you have the necessary skills and resources? 
    R->Relevant: How does the goal align with broad

In [108]:
# Now take the learning plan find relevant Jobs and extract the skills required for the job
learning_plan


{'specific': 'To become a SAP consultant, I will gain in-depth knowledge of SAP software and its various modules, as well as develop strong technical and consulting skills. I will work towards obtaining SAP certifications to demonstrate my expertise.',
 'measurable': 'I will measure my progress by successfully completing SAP certification exams, such as the SAP Certified Application Associate certification, and by taking on increasingly complex SAP-related projects and tasks at work. I will also track the number of SAP-related job opportunities I am qualified for and the number of interviews I receive.',
 'achievable': 'Becoming a SAP consultant is an achievable goal for me, as I have a strong background in computer science and programming, as well as experience working with enterprise software. I have the necessary technical skills and resources, such as access to SAP training materials and the ability to enroll in SAP certification courses.',
 'relevant': 'Becoming a SAP consultant a

In [109]:
context = response = client.retrieve(
    knowledgeBaseId=AWS_KB_ID,
     retrievalQuery={
        'text': learning_plan["specific"] + learning_plan["measurable"],
    }
)


In [113]:
for result in context["retrievalResults"]:
    job_desc = result["content"]["text"]
    print(job_desc)
    get_skills(job_desc)

Build strong, trusting relationships with your clients’ senior leaders to help Accenture become the go-to partner in bringing their long-term fashion or retail vision to life Grow Accenture’s SAP practice by leading business development, proposal submission, and sales efforts Serve as an expert and thought leader in the Accenture SAP practice by continuing to build your own and your team’s expertise in SAP Travel may be required for this role. The amount of travel will vary from 0 to 100% depending on business need and client requirements Here’s What You Need: Minimum 8 years of experience in SAP Fashion, SAP Retail, and/or SAP CAR Minimum 3 end-to-end SAP fashion or retail implementations, including project planning, estimation and solution architecture Minimum 6 years of experience in Project Management Experience managing teams of 15+, including offshore teams Bachelor's degree or equivalent (minimum 12 years' work experience).

You are a career counselor. Your task is to extract sk

## Stage 6: Agent Testing

In [155]:
memory = SqliteSaver.from_conn_string(":memory:")
model = ChatBedrock(model_id ="anthropic.claude-3-haiku-20240307-v1:0", model_kwargs={"temperature": 0, "max_tokens":6000, "top_p":0.1,"top_k":50})
tools = [extract_smart_goal, get_skills]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [160]:
q1  = f"Here is my resume {resume_text}, Can you extract my learing goal from my aspiration: {ASPIRATION}?"
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
{"messages": [HumanMessage(content=q1)]}, config
):
    print(chunk)
    print("----")

ValueError: Error raised by bedrock service: An error occurred (ValidationException) when calling the InvokeModel operation: messages.6: Did not find 1 `tool_result` block(s) at the beginning of this message. Messages following `tool_use` blocks must begin with a matching number of `tool_result` blocks.

In [157]:
q1  = f"My aspiration is {ASPIRATION}. Can you help me create a learning plan?"
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
{"messages": [HumanMessage(content=q1)]}, config
):
    print(chunk)
    print("----")


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'usage': {'prompt_tokens': 3136, 'completion_tokens': 79, 'total_tokens': 3215}, 'stop_reason': 'tool_use', 'model_id': 'anthropic.claude-3-haiku-20240307-v1:0'}, response_metadata={'usage': {'prompt_tokens': 3136, 'completion_tokens': 79, 'total_tokens': 3215}, 'stop_reason': 'tool_use', 'model_id': 'anthropic.claude-3-haiku-20240307-v1:0'}, id='run-f7ba49d8-949e-45b6-9487-d8282d371599-0', tool_calls=[{'name': 'get_skills', 'args': {'specific_goal': 'Become a SAP consultant'}, 'id': 'toolu_bdrk_01BkykikoradYP5rqeddpt9L', 'type': 'tool_call'}], usage_metadata={'input_tokens': 3136, 'output_tokens': 79, 'total_tokens': 3215})]}}
----
Build strong, trusting relationships with your clients’ senior leaders to help Accenture become the go-to partner in bringing their long-term fashion or retail vision to life Grow Accenture’s SAP practice by leading business development, proposal submission, and sales efforts Serve as an exper

KeyboardInterrupt: 

In [142]:
learning_plan

{'specific': 'To become a SAP consultant, I will gain in-depth knowledge of SAP software and its various modules, as well as develop strong technical and consulting skills. I will work towards obtaining SAP certifications to demonstrate my expertise.',
 'measurable': 'I will measure my progress by successfully completing SAP certification exams, such as the SAP Certified Application Associate certification, and by taking on increasingly complex SAP-related projects and tasks at work. I will also track the number of SAP-related job opportunities I am qualified for and the number of interviews I receive.',
 'achievable': 'Becoming a SAP consultant is an achievable goal for me, as I have a strong background in computer science and programming, as well as experience working with enterprise software. I have the necessary technical skills and resources, such as access to SAP training materials and the ability to enroll in SAP certification courses.',
 'relevant': 'Becoming a SAP consultant a

In [143]:
q2 = f"Can you help me extract the skills that might be required for my learning plan?"
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
{"messages": [HumanMessage(content=q2)]}, config
):
    print(chunk)
    print("----")


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'usage': {'prompt_tokens': 6117, 'completion_tokens': 127, 'total_tokens': 6244}, 'stop_reason': 'tool_use', 'model_id': 'anthropic.claude-3-haiku-20240307-v1:0'}, response_metadata={'usage': {'prompt_tokens': 6117, 'completion_tokens': 127, 'total_tokens': 6244}, 'stop_reason': 'tool_use', 'model_id': 'anthropic.claude-3-haiku-20240307-v1:0'}, id='run-f5b292fe-9d82-4291-8740-93c251a884cf-0', tool_calls=[{'name': 'get_skills', 'args': {'specific_goal': 'To become a proficient SAP consultant within the next 2 years. This will involve gaining in-depth knowledge of SAP modules, obtaining relevant SAP certifications, and gaining hands-on experience through internships or projects.'}, 'id': 'toolu_bdrk_01LMV2w2eZ6wyd6yQqbTdCZQ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 6117, 'output_tokens': 127, 'total_tokens': 6244})]}}
----
Build strong, trusting relationships with your clients’ senior leaders to help Accentur

In [145]:
q3 = f"Can you tell me why SAP expertise is required for my learning plan?, give me a short response"
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
{"messages": [HumanMessage(content=q3)]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='SAP expertise is required for your learning plan because it is the core competency needed to become a proficient SAP consultant. As an SAP consultant, you need in-depth knowledge of SAP modules, hands-on experience with implementation and configuration, and proficiency in SAP-specific technologies. This expertise allows you to provide thought leadership, innovate solutions, and build credibility with clients.', additional_kwargs={'usage': {'prompt_tokens': 11372, 'completion_tokens': 85, 'total_tokens': 11457}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-haiku-20240307-v1:0'}, response_metadata={'usage': {'prompt_tokens': 11372, 'completion_tokens': 85, 'total_tokens': 11457}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-haiku-20240307-v1:0'}, id='run-cd992103-697c-470a-9340-2cc8376d52e0-0', usage_metadata={'input_tokens': 11372, 'output_tokens': 85, 'total_tokens': 11457})]}}
----
